In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Load the dataset
df = pd.read_csv("ecommerce_sales.csv")

# Step 1: Data Cleaning
# Remove duplicates
df.drop_duplicates(inplace=True)

# Handle missing values
df.dropna(inplace=True)

# Convert InvoiceDate to datetime format
if 'InvoiceDate' in df.columns:
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Remove negative values (assuming 'Quantity' and 'UnitPrice' should be positive)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Save cleaned data
df.to_csv("cleaned_data.csv", index=False)

# Step 2: Exploratory Data Analysis (EDA)
# Basic Info
print(df.info())
print(df.describe())

# Univariate Analysis
plt.figure(figsize=(12, 6))
df.hist(figsize=(12, 6), bins=30)
plt.show()

# Boxplot for UnitPrice
plt.figure(figsize=(8,5))
sns.boxplot(x=df['UnitPrice'])
plt.title("Unit Price Distribution")
plt.show()

# Top Selling Products
print(df['Description'].value_counts().head(10))

# Orders per Country
plt.figure(figsize=(12, 6))
df['Country'].value_counts().plot(kind='bar')
plt.title("Orders per Country")
plt.xlabel("Country")
plt.ylabel("Count")
plt.xticks(rotation=90)
plt.show()

# Sales Trend Over Time
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df.groupby("Month")['UnitPrice'].sum().plot(kind="line", marker="o", figsize=(10, 5))
plt.title("Monthly Sales Trend")
plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.grid()
plt.show()

# Correlation Analysis
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

# Step 3: SQL Analysis
conn = sqlite3.connect("ecommerce.db")
df.to_sql("sales", conn, if_exists="replace", index=False)

# Query 1: Total Sales by Country
query = "SELECT Country, SUM(UnitPrice * Quantity) AS TotalSales FROM sales GROUP BY Country ORDER BY TotalSales DESC;"
result = pd.read_sql(query, conn)
print(result)

# Query 2: Top 5 Best Selling Products
query = "SELECT Description, SUM(Quantity) AS TotalQuantity FROM sales GROUP BY Description ORDER BY TotalQuantity DESC LIMIT 5;"
result = pd.read_sql(query, conn)
print(result)

# Query 3: Monthly Sales Trend
query = "SELECT strftime('%Y-%m', InvoiceDate) AS Month, SUM(UnitPrice * Quantity) AS TotalSales FROM sales GROUP BY Month ORDER BY Month;"
result = pd.read_sql(query, conn)
print(result)

# Close the connection
conn.close()

print("Data Cleaning, EDA, and SQL Analysis Completed Successfully!")


ModuleNotFoundError: No module named 'numpy'